<a href="https://colab.research.google.com/github/SanjayS2348553/Trimester-3/blob/main/2348553_DL_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>